In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pandas as pd
from os import getcwd
from os.path import join
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../data/gifts.csv')

In [ ]:
df

# Dataframe statistics

In [ ]:
df.describe()

In [ ]:

print("Number of rows : ", len(df))
print(" ")
print("Min Weight: ", df.Weight.min())
print("Max Weight: ", df.Weight.max())

print("Min Longitude: ",df.Longitude.min())
print("Max Longitude: ",df.Longitude.max())
print("Min Latitude: ",df.Latitude.min())
print("Max Latitude: ",df.Latitude.max())

plt.rcParams['figure.figsize'] = 20, 10
plt.hist2d(df.Longitude, df.Latitude, bins=360)
cb = plt.colorbar()
cb.set_label("density")

In [ ]:
plt.hist(df['Weight'], bins = 30)

# Harvesine Distance

- extract lat & lon
- stack values into np.array
- reshape np.array to (-1,2)
- convert lat & lon to radians
- compute pairwise harvesine distance using sklearn


## Basic examples (Sklearn)

### 1. using lists

In [ ]:
from sklearn.metrics.pairwise import haversine_distances
from math import radians

bsas = [-34.83333, -58.5166646]
paris = [49.0083899664, 2.53844117956]
london = [51.5074, 0.1278]

bsas_in_radians = [radians(_) for _ in bsas]
paris_in_radians = [radians(_) for _ in paris]
london_in_radians = [radians(_) for _ in london]

print(bsas_in_radians)
print(paris_in_radians)
print(london_in_radians)
result = haversine_distances([bsas_in_radians, paris_in_radians,london_in_radians])
result * 6371000/1000  # multiply by Earth radius to get kilometers

### 2. using np.arrays

In [ ]:
bsas = [-34.83333, -58.5166646]
paris = [49.0083899664, 2.53844117956]
london = [51.5074, 0.1278]

cities = np.array([bsas,paris,london])
cities_radian = np.radians(cities)
result = haversine_distances(cities_radian)
result * 6371000/1000  # multiply by Earth radius to get kilometers

### Pandas and Numpy

In [ ]:
# The first number is always the latitude and the second is the longitude ;)
coords = df[['Latitude','Longitude']].head(10).to_numpy()
coords = np.radians(coords)
result = haversine_distances(coords)
result * 6371000/1000  # multiply by Earth radius to get kilomet
#df['Latitude'] = coords[:,0]
#df['Longitude'] = coords[:,1]

## Why Not to calculate pairwise distance between all locations..
- The adj. Matrix of the complet craph would need n**2 * np.float memory
- As calculated below, this would not be wise :D but for each tour it would be no problem (mean weight = 14.xx --> 70 stops on a maximum capacity of 1000) 
- Nevertheless, it could be useful to just transform the coordinates into radians anyway. That way, it wouldn't be necessary to do the calculations each time a sample is used for the weighted reindeer wearniess...

In [ ]:
adj_mat_size = len(df)
print("Adj Matrix dimension: (" ,adj_mat_size,",",adj_mat_size,")")
print("Memory needed (GB):", (adj_mat_size**2)*8/10**9 ) # roughly 8 bytes for a float in numpy arrays

# Weighted reindeer weariness

$$WRW = \sum\limits_{j=1}^{m} \sum\limits_{i=1}^{n} \Big[ \big( \sum\limits_{k=1}^{n} w_{kj} - \sum\limits_{k=1}^{i} w_{kj} \big) \cdot Dist(Loc_i, Loc_{i-1}) \Big]_j ,$$

$$m := \text{number of trips} $$
$$j := \text{one specific trip}$$
$$n := \text{nmber of gifts (per trip j) }$$
$$w_{ij} := \text{weight of the }i^{th} \text{ gift at trip j}$$ 
$$Loc_{0}\text{ and } Loc_{0} \text{is the north pole for each trip j}$$ 
$$w_{nj} := \text{is the weight of the empty sled}$$ 

## Mini Example
Example with the first ten entries:
- trip 1 = entries 0:4
- trip 2 = entries 5:9

trip1 : North_Pole --> 0 --> 1 --> 2 --> 3 --> 4 --> North_Pole\
trip2 : North_Pole --> 5 --> 6 --> 7 --> 8 --> 9 --> North_Pole

In [ ]:
def weighted_reindeer_weariness(trips):
    weighted_weariness = 0
    for trip in trips:
        weights = trip['Weight'].to_numpy()
        coordinates = trip[['Latitude','Longitude']].to_numpy()
        weighted_weariness = weighted_weariness + weighted_distance(coordinates,weights,sleigh_weight)
    return weighted_weariness
    
def weighted_distance(coordinates,weights,sleigh_weight):
    startweight = sleigh_weight + np.sum(weights)
    if startweight > weight_limit:
        return -1

    north_pole = np.radians([90,0])
    coords = np.vstack((north_pole,coordinates,north_pole))
  
    adj_matrix = haversine_distances(coords,np.roll(coords.copy(),-1,axis=0))
    adj_matrix = adj_matrix * 6371 #6371000/1000
    distances = np.diag(adj_matrix)[:-1]
    
    weights +=sleigh_weight
    weights = np.append(weights,sleigh_weight)
    weights = np.cumsum(weights[::-1])[::-1] # flip, cummulative sum, flip again

    """
    print(coords)
    for i in range(len(coords)-1):
        print(haversine_distances([coords[i],coords[i+1]])[0][1]*6371)
    
    with np.printoptions(precision=3, suppress=True):
        print(distances,2)
    """
    weighted_dist = np.sum(weights*distances)
    print("weighted_dist ",weighted_dist)


    return weighted_dist


In [ ]:
    
weight_limit = 1000
sleigh_weight = 10

entries = df.head(10)
trip1 = entries[:5].copy()
trip2 = entries[5:].copy()

trips = [trip1.copy(),trip2.copy()]

WRW = weighted_reindeer_weariness(trips)
print("Total Wariness: ", WRW)


# Graph Implementation
 This is a naive implementation of a graph assigning each of the n coordinate for a gift delivery as a unique tour id.
- obviously it results in n tours which is far from optimal
- the TourId is stored as a new column in the dataframe
- I guess this could be a starting point for building a solution

In [ ]:
class Graph:
    def __init__(self, gifts):
        self.numEdges = 0
        data = self.init_tours(gifts)
        self.tourgraph = pd.DataFrame(data)
        
    def init_tours(self, gifts):
        tripIds = np.arange(len(gifts))
        gifts_copy = gifts.copy()
        gifts_copy['TripId'] = tripIds
        return gifts_copy
    
    def weighted_reindeer_weariness(self, trips):
        weighted_weariness = 0
        grouped_trips = trips.groupby('TripId')
    
        for group_name, trip in grouped_trips:
            weights = trip['Weight'].to_numpy()
            coordinates = trip[['Latitude','Longitude']].to_numpy()
            weighted_weariness = weighted_weariness + self.weighted_distance(coordinates,weights,sleigh_weight)
        #print("weighted_weariness ",weighted_weariness)
        return weighted_weariness
    
    def weighted_distance(self, coordinates, weights,sleigh_weight):
        startweight = sleigh_weight + np.sum(weights)
        if startweight > weight_limit:
            return -1

        north_pole = np.radians([90,0])
        coords = np.vstack((north_pole,coordinates,north_pole))

        adj_matrix = haversine_distances(coords,np.roll(coords.copy(),-1,axis=0))
        adj_matrix = adj_matrix * 6371 #6371000/1000
        distances = np.diag(adj_matrix)[:-1]

        weights +=sleigh_weight
        weights = np.append(weights,sleigh_weight)
        weights = np.cumsum(weights[::-1])[::-1] # flip, cummulative sum, flip again

        weighted_dist = np.sum(weights*distances)
        return weighted_dist

    
graph = Graph(df.head(10))
tours = graph.tourgraph      

In [ ]:
tours

In [ ]:

graph.weighted_reindeer_weariness(tours)


# Clustering

In [ ]:
df = pd.read_csv('../data/gifts.csv')
offset = 0.5
slices = []
for i in range(0,360):
    j = i+0.5
    slices.append(df[(df['Longitude']>(j-offset)) & (df['Longitude']<(j+offset))])
    
for s in slices[:5]:
    s.plot.scatter('Longitude','Latitude')
    s.describe()

first_slice = slices[0]

In [ ]:
from sklearn.cluster import KMeans
import seaborn as sns
import numpy as np
X = first_slice[['Latitude','Longitude']]
kmeans = KMeans(n_clusters=10, random_state=0).fit(X)
labels = kmeans.labels_
first_slice['cluster_labels'] = kmeans.labels_

centers = kmeans.cluster_centers_

groups = first_slice.groupby('cluster_labels')
for group_name, group in groups:
    sum_weight = group['Weight'].sum()
    print("sum_weight ", sum_weight)



s.plot.scatter('Longitude','Latitude')
s.describe()
fig, ax = plt.subplots(figsize=(20,8))
sns.scatterplot(data=first_slice, x="Longitude", y="Latitude", hue='cluster_labels', size=2)



# Slicing and initial Tours

In [ ]:
df = pd.read_csv('../data/gifts.csv')
offset = 0.5
slices = []

for i in range(-180,181):
    j = i+0.5
    slices.append(df[(df['Longitude']>=(j-offset)) & (df['Longitude']<(j+offset))])


In [ ]:
import functools as ft
tours = []
tripId = 0

tour = {"GiftId" : [],"Latitude" : [],"Longitude" :[],"Weight" : [],"TripId":[]}
counter = 0
for s in slices:
    print(len(s))
    for index,row in s.iterrows():
        counter = counter + 1
        sum_current_tour = ft.reduce(lambda x,y:x+y,tour['Weight'],0) 
        if (row['GiftId']==38333):
            print("------BAD------")
        if (sum_current_tour+row['Weight'])<=weight_limit:
            if (row['GiftId']==38333):
                print("------BAD IF------")
            tour['GiftId'].append(row['GiftId'])
            tour['Latitude'].append(row['Latitude'])
            tour['Longitude'].append(row['Longitude'])
            tour['Weight'].append(row['Weight']) 
            tour['TripId'].append(tripId) 
        else:
            tripId +=1
            sum_current_tour = 0
            tours.append(tour.copy())
            tour = {"GiftId" : [],"Latitude" : [],"Longitude" :[],"Weight" : [],"TripId":[]}
            tour['GiftId'].append(row['GiftId'])
            tour['Latitude'].append(row['Latitude'])
            tour['Longitude'].append(row['Longitude'])
            tour['Weight'].append(row['Weight']) 
            tour['TripId'].append(tripId) 
tours.append(tour.copy())

            
print(counter)


In [ ]:
#def concat_dicts(dict1,dict2):
from collections import ChainMap

res = {} 
for dict in tours: 
    for list in dict: 
        if list in res: 
            res[list] += (dict[list]) 
        else: 
            res[list] = dict[list] 
print(len(pd.DataFrame(res)))
print(len(df))

In [ ]:
tourIds = []
for tour in tours:
    tourIds.append(tour['TripId'])


graph = Graph(df.head(10))
tours = graph.tourgraph   
graph.weighted_reindeer_weariness(tours)

In [ ]:
l = 0
for tour in tours:
    l += len(tour['GiftId'])
print(l)